## Libraries


In [7]:
import pandas as pd
from src.feature_engineering import features
import numpy as np

## Data


In [2]:
geral_and_comunicados_df = pd.read_csv("data/portfolio_geral_comunicados.csv")
geral_and_comunicados_df

,contrato_id,dt_ref_portfolio,safra,nr_documento,status_contrato,dt_contrato,dt_desembolso,dt_vencimento,dt_wo,prazo,...,vlr_pgto_esperado,vlr_saldo_devedor,vlr_saldo_devedor_esperado,dsp,dspp,flag_transacao,data_acao,tipo_acao,status,acao
0,4207c25cdd13aba5a05ae348e889d798,2022-03-01,2021-03,9c1dc4403f8a4833bd58eea3ee85c917,Active,2021-03-01,2021-03-03,2022-06-01,2023-06-01,25.65,...,43.06,9469.47,461.96,20,82,0,NaN,NaN,NaN,NaN
1,7dfdca96b6ff0dd844e5e130aee88b30,2022-03-01,2020-11,3b6951df3075979c99a4dfd9f93efd38,Active,2020-10-31,2020-11-04,2022-01-31,2023-01-31,25.65,...,200.09,57783.55,0.00,134,167,0,NaN,NaN,NaN,NaN
2,a3aecc6b029cc77dd867e0e24e2d616f,2022-03-01,2020-12,9740eda3d6190add5703cd2dd280023c,Active,2020-12-18,2020-12-22,2022-06-18,2023-06-18,30.78,...,89.99,36764.23,2745.73,196,250,0,NaN,NaN,NaN,NaN
3,3db18b4a0e36098c1ef1ee87f1491d84,2022-03-01,2020-07,c2766a0c65d1df592aeb4797efc893e0,Active,2020-07-08,2020-07-10,2021-10-08,2022-10-08,25.65,...,82.61,66762.40,0.00,286,594,0,NaN,NaN,NaN,NaN
4,b4658a6d9785c527297cd14762240f09,2022-03-01,2020-12,d61e4f520ee20e9c693c9c4808d19f55,Active,2020-12-04,2020-12-08,2022-03-04,2023-03-04,25.65,...,131.89,20054.13,0.00,76,76,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6647552,7972c2833e109f498ad29656dcf83c50,2020-11-12,2020-09,ce37f25645e7460fdc44a8f9f6aa46a9,Active,2020-09-29,2020-09-30,2022-03-29,2023-03-29,30.78,...,36.70,10342.73,10669.01,3,6,1,NaN,NaN,NaN,NaN
6647553,b532ce3583bf083b58dcbb3156e9a6fb,2020-10-04,2020-09,66a443539af19dc890cafbebe0e1de8e,Active,2020-09-25,2020-09-29,2021-12-25,2022-12-25,25.65,...,35.41,11201.47,11229.76,2,2,0,NaN,NaN,NaN,NaN
6647554,c5cce1cc49025572c7378279c1469923,2020-10-04,2020-05,f9d63ef575f20c5698146e2c796eedcd,Settled,2020-05-26,2020-05-29,2021-08-26,2022-08-26,25.65,...,9.91,0.02,2756.18,0,0,0,NaN,NaN,NaN,NaN
6647555,67348b340a082aa77a67b7431cc91145,2020-10-31,2020-06,28283b4f3cc1654a879b64251f98164b,Active,2020-06-17,2020-06-23,2021-09-17,2022-09-17,25.65,...,5.64,2064.82,1342.64,122,122,0,NaN,NaN,NaN,NaN


In [3]:
# datatypes
geral_and_comunicados_df.dtypes

contrato_id                    object
dt_ref_portfolio               object
safra                          object
nr_documento                   object
status_contrato                object
dt_contrato                    object
dt_desembolso                  object
dt_vencimento                  object
dt_wo                          object
prazo                         float64
vlr_desembolsado              float64
vlr_tarifa                    float64
juros_mes                     float64
juros_diario                  float64
perc_retencao                 float64
vlr_pgto_realizado            float64
vlr_pgto_esperado             float64
vlr_saldo_devedor             float64
vlr_saldo_devedor_esperado    float64
dsp                             int64
dspp                            int64
flag_transacao                  int64
data_acao                      object
tipo_acao                      object
status                         object
acao                           object
dtype: objec

## Análise


Aqui iremos primeiramente criar um target que descreverar quando que cada comunicado foi efetivo, ou não, na conversão de um inadiplente para alguém que retornou a realizar pagamentos.

A ideia é que para cada contrato, seja possível sumerizar métricas de sucesso ou não de todas as comunicações efetuadas. Como a intenção é que o cliente volte a realizar pagamentos a partir de ter recebido/lido uma notificação, iremos buscar então, os momentos em que o dsp e o dspp zeram, considerando todo o histórico de cada contrato individualmente.

Abaixo veremos a criação dessa feature.


In [4]:
geral_and_comunicados_sorted_df = geral_and_comunicados_df.sort_values(
    ["contrato_id", "dt_ref_portfolio"]
)

In [43]:
# geral_and_comunicados_df.value_counts(["status_contrato"])
geral_and_comunicados_df[
    geral_and_comunicados_df["contrato_id"] == "c3d178c6993f07f4af3897a8ed64793c"
].sort_values(["dt_ref_portfolio"])
# geral_and_comunicados_df[geral_and_comunicados_df["status_contrato"] == "Suspended"]

,contrato_id,dt_ref_portfolio,safra,nr_documento,status_contrato,dt_contrato,dt_desembolso,dt_vencimento,dt_wo,prazo,...,vlr_pgto_esperado,vlr_saldo_devedor,vlr_saldo_devedor_esperado,dsp,dspp,flag_transacao,data_acao,tipo_acao,status,acao
6257490,c3d178c6993f07f4af3897a8ed64793c,2020-08-19,2020-08,7a132245f1731fdb6568cfaa13772b8a,Active,2020-08-16,2020-08-19,2021-11-16,2022-11-16,25.65,...,81.17,27792.63,27747.73,0,0,0,NaN,NaN,NaN,NaN
1951591,c3d178c6993f07f4af3897a8ed64793c,2020-08-20,2020-08,7a132245f1731fdb6568cfaa13772b8a,Active,2020-08-16,2020-08-19,2021-11-16,2022-11-16,25.65,...,81.17,27828.90,27702.77,1,1,1,NaN,NaN,NaN,NaN
1917461,c3d178c6993f07f4af3897a8ed64793c,2020-08-21,2020-08,7a132245f1731fdb6568cfaa13772b8a,Active,2020-08-16,2020-08-19,2021-11-16,2022-11-16,25.65,...,81.17,27770.71,27657.75,0,0,1,NaN,NaN,NaN,NaN
5859051,c3d178c6993f07f4af3897a8ed64793c,2020-08-22,2020-08,7a132245f1731fdb6568cfaa13772b8a,Active,2020-08-16,2020-08-19,2021-11-16,2022-11-16,25.65,...,81.17,27806.94,27612.67,1,1,1,NaN,NaN,NaN,NaN
6605463,c3d178c6993f07f4af3897a8ed64793c,2020-08-23,2020-08,7a132245f1731fdb6568cfaa13772b8a,Active,2020-08-16,2020-08-19,2021-11-16,2022-11-16,25.65,...,81.17,27843.23,27567.54,2,2,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6233543,c3d178c6993f07f4af3897a8ed64793c,2022-04-14,2020-08,7a132245f1731fdb6568cfaa13772b8a,Settled,2020-08-16,2020-08-19,2021-11-16,2022-11-16,25.65,...,81.17,0.00,0.00,0,0,0,NaN,NaN,NaN,NaN
2893454,c3d178c6993f07f4af3897a8ed64793c,2022-04-15,2020-08,7a132245f1731fdb6568cfaa13772b8a,Settled,2020-08-16,2020-08-19,2021-11-16,2022-11-16,25.65,...,81.17,0.00,0.00,0,0,0,NaN,NaN,NaN,NaN
1726556,c3d178c6993f07f4af3897a8ed64793c,2022-04-16,2020-08,7a132245f1731fdb6568cfaa13772b8a,Settled,2020-08-16,2020-08-19,2021-11-16,2022-11-16,25.65,...,81.17,0.00,0.00,0,0,0,NaN,NaN,NaN,NaN
5832346,c3d178c6993f07f4af3897a8ed64793c,2022-04-17,2020-08,7a132245f1731fdb6568cfaa13772b8a,Settled,2020-08-16,2020-08-19,2021-11-16,2022-11-16,25.65,...,81.17,0.00,0.00,0,0,0,NaN,NaN,NaN,NaN


In [5]:
contrato_v2 = (
    geral_and_comunicados_sorted_df.groupby(["contrato_id"])["dsp"]
    .agg(
        [
            features.total_success_dsp5,
            features.total_success_dsp10,
            features.total_success_dsp15,
            features.total_success_dsp30,
            features.total_success_dsp60,
            features.total_success_dsp90,
        ]
    )
    .reset_index()
)

In [8]:
means = []
for i, row in contrato_v2.iterrows():
    means.append(np.nanmean(row[1:]))

contrato_v2["means"] = means

/tmp/ipykernel_1157832/3896819483.py:3: RuntimeWarning: Mean of empty slice
  means.append(np.nanmean(row[1:]))


In [ ]:
contrato_v2.sort_values(["means"], ascending=False).head(20)

,contrato_id,total_success_dsp5,total_success_dsp10,total_success_dsp15,total_success_dsp30,total_success_dsp60,total_success_dsp90,means
7761,a20e2ee4f3f206dd5dab3dc1ab17100f,1.0,NaN,NaN,NaN,NaN,NaN,1.0
5775,79705b6a4d46b1b4f7f6293711a4bebf,1.0,NaN,NaN,NaN,NaN,NaN,1.0
10700,e07e02b6fc63f58258db365ffc9dc81e,1.0,NaN,NaN,NaN,NaN,NaN,1.0
8912,ba3c8dafef632e71dd2b0cf45afbc96f,1.0,NaN,NaN,NaN,NaN,NaN,1.0
2726,3a67a6106724ce2e884cede96680a6bf,1.0,NaN,NaN,NaN,NaN,NaN,1.0
2730,3a92b79f46012997689b5e63375071a1,1.0,NaN,NaN,NaN,NaN,NaN,1.0
5877,7b9f9bd7e7806d98f53b00bec3364cbf,1.0,NaN,NaN,NaN,NaN,NaN,1.0
2738,3ab698f751c87943e46958596986e9dc,1.0,NaN,NaN,NaN,NaN,NaN,1.0
8919,ba4537d10d30403cff31a5eaca8f8bab,1.0,NaN,NaN,NaN,NaN,NaN,1.0
10690,e060e321ef1201f65df6120efef969c7,1.0,NaN,NaN,NaN,NaN,NaN,1.0


In [19]:
nr_documentos = []
for contrato_id in contrato_v2["contrato_id"]:
    # contrato_v2["nr_documento"] =
    nr_documentos.append(
        geral_and_comunicados_df[
            geral_and_comunicados_df["contrato_id"] == contrato_id
        ]["nr_documento"].values[0]
    )

# contrato_v2

In [25]:
p_cliente = pd.read_csv("data/portfolio_clientes.csv")
p_cliente

,nr_documento,tipo_empresa,cidade,estado,subsegmento,segmento
0,094aaed740bbd7fcddb6c678c18d883a,PJ,Caucaia,CE,Alimentação Rápida,Alimentação
1,eaade7915611fcdcbba747e3a83365c6,PF,Cidade Ocidental,GO,Lojas Diversas,Varejo
2,1f21958fd296f1b04a1f0c2658913856,MEI,Manaus,AM,Alimentação Rápida,Alimentação
3,fcdfe15db4520db1a1fe8b6a72878655,MEI,São Bernardo do Campo,SP,Alimentação Rápida,Alimentação
4,ba57e8837452b5e0114d7e273de039b4,PJ,Foz do Iguaçu,PR,Lojas Diversas,Supermercado/Farmácia
...,...,...,...,...,...,...
14400,bfed2a37a79957ea53cf1b70d467e4f1,PJ,Bage,RS,Outros,Serviços
14401,fbcba6437b248dfce69ac494b4c266f1,MEI,Osasco,SP,Automotivo,Bens duráveis
14402,29b16c64a0d582d0162c8346bcbab1b0,PJ,São Carlos,SP,Alimentação Rápida,Alimentação
14403,2ba5b641cad668157ede5a32fe1f3b00,MEI,Imbituba,SC,Educação,Serviços recorrentes


In [28]:
contrato_v2_merged = contrato_v2.merge(right=p_cliente, on="nr_documento", how="left")

In [21]:
contrato_v2["nr_documento"] = nr_documentos

In [23]:
contrato_v2.to_csv("data/contrato_v2.csv", index=False)

In [31]:
contrato_v2_merged.groupby("segmento")["means"].agg([np.nanmean, np.nanstd])

,nanmean,nanstd
segmento,,
Alimentação,0.562365,0.338842
Bens duráveis,0.609768,0.310627
Outros,0.510062,0.294551
Posto,0.561797,0.346220
Serviços,0.631668,0.306019
Serviços recorrentes,0.663638,0.284040
Supermercado/Farmácia,0.524403,0.341507
Varejo,0.584366,0.323720
Viagens e entretenimento,0.609646,0.316411


In [32]:
contrato_v2_merged.groupby("subsegmento")["means"].agg([np.nanmean, np.nanstd])

,nanmean,nanstd
subsegmento,,
Academias,0.643048,0.291416
Alimentação Rápida,0.577527,0.337971
Atacadista de Alimento,0.537074,0.300768
Atacadistas Gerais,0.608007,0.300267
Automotivo,0.658932,0.303395
Bares e Restaurantes,0.566843,0.330761
Educação,0.676000,0.278203
Lazer & Turismo,0.601376,0.290022
Lojas Diversas,0.566411,0.336482


In [35]:
ali = contrato_v2_merged[contrato_v2_merged["segmento"] == "Alimentação"]

In [ ]:
m_array = np.array([0, 1, np.nan])
np.nanmean(m_array)

0.5

In [ ]:
p_cliente.value_counts(["segmento", "subsegmento"]).reset_index().sort_values(
    "segmento"
)

,segmento,subsegmento,0
0,Alimentação,Alimentação Rápida,2959
25,Alimentação,Atacadista de Alimento,50
3,Alimentação,Bares e Restaurantes,1234
6,Alimentação,Lojas Diversas,586
11,Alimentação,Supermercados,387
20,Bens duráveis,Atacadistas Gerais,118
36,Bens duráveis,Lazer & Turismo,1
7,Bens duráveis,Automotivo,567
8,Bens duráveis,Lojas Diversas,566
16,Bens duráveis,Saúde,202


In [ ]:
p_cliente.value_counts(["subsegmento"])

subsegmento           
Alimentação Rápida        2959
Lojas Diversas            2710
Outros                    1590
Bares e Restaurantes      1306
Supermercados             1198
Automotivo                1107
Vestuário                 1014
Saúde                      836
Material de Construção     540
Atacadistas Gerais         416
Educação                   193
Postos de Gasolina         166
Academias                  117
Lazer & Turismo            117
None                        86
Atacadista de Alimento      50
dtype: int64

In [ ]:
# code for an example only
two_contrato = geral_and_comunicados_df[
    geral_and_comunicados_df["contrato_id"].isin(
        ["4207c25cdd13aba5a05ae348e889d798", "7dfdca96b6ff0dd844e5e130aee88b30"]
    )
]
two_contrato

,contrato_id,dt_ref_portfolio,safra,nr_documento,status_contrato,dt_contrato,dt_desembolso,dt_vencimento,dt_wo,prazo,...,vlr_pgto_esperado,vlr_saldo_devedor,vlr_saldo_devedor_esperado,dsp,dspp,flag_transacao,data_acao,tipo_acao,status,acao
0,4207c25cdd13aba5a05ae348e889d798,2022-03-01,2021-03,9c1dc4403f8a4833bd58eea3ee85c917,Active,2021-03-01,2021-03-03,2022-06-01,2023-06-01,25.65,...,43.06,9469.47,461.96,20,82,0,NaN,NaN,NaN,NaN
1,7dfdca96b6ff0dd844e5e130aee88b30,2022-03-01,2020-11,3b6951df3075979c99a4dfd9f93efd38,Active,2020-10-31,2020-11-04,2022-01-31,2023-01-31,25.65,...,200.09,57783.55,0.00,134,167,0,NaN,NaN,NaN,NaN
7,7dfdca96b6ff0dd844e5e130aee88b30,2022-03-24,2020-11,3b6951df3075979c99a4dfd9f93efd38,Active,2020-10-31,2020-11-04,2022-01-31,2023-01-31,25.65,...,200.09,59543.06,0.00,157,190,0,NaN,NaN,NaN,NaN
78,4207c25cdd13aba5a05ae348e889d798,2022-02-04,2021-03,9c1dc4403f8a4833bd58eea3ee85c917,Active,2021-03-01,2021-03-03,2022-06-01,2023-06-01,25.65,...,43.06,9230.67,1537.05,38,57,0,NaN,NaN,NaN,NaN
150,4207c25cdd13aba5a05ae348e889d798,2021-06-20,2021-03,9c1dc4403f8a4833bd58eea3ee85c917,Active,2021-03-01,2021-03-03,2022-06-01,2023-06-01,25.65,...,43.06,12241.70,11317.65,4,9,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6639174,7dfdca96b6ff0dd844e5e130aee88b30,2021-12-12,2020-11,3b6951df3075979c99a4dfd9f93efd38,Active,2020-10-31,2020-11-04,2022-01-31,2023-01-31,25.65,...,200.09,52126.63,1994.25,55,88,0,NaN,NaN,NaN,NaN
6640136,7dfdca96b6ff0dd844e5e130aee88b30,2021-03-12,2020-11,3b6951df3075979c99a4dfd9f93efd38,Active,2020-10-31,2020-11-04,2022-01-31,2023-01-31,25.65,...,200.09,54630.36,52840.06,0,3,1,NaN,NaN,NaN,NaN
6640284,7dfdca96b6ff0dd844e5e130aee88b30,2021-07-23,2020-11,3b6951df3075979c99a4dfd9f93efd38,Active,2020-10-31,2020-11-04,2022-01-31,2023-01-31,25.65,...,200.09,50287.66,30377.13,0,0,1,NaN,NaN,NaN,NaN
6641296,4207c25cdd13aba5a05ae348e889d798,2022-04-08,2021-03,9c1dc4403f8a4833bd58eea3ee85c917,Active,2021-03-01,2021-03-03,2022-06-01,2023-06-01,25.65,...,42.70,9290.60,0.00,9,120,0,NaN,NaN,NaN,NaN


In [ ]:
two_contrato_sorted = two_contrato.sort_values(["contrato_id", "dt_ref_portfolio"])

In [ ]:
example = [1, 2, 3, 4, 5, 6, 7, 8]

for i, x in enumerate(example):
    if x == 2:
        new_list = example[i : i + 7]
        for item in new_list:
            print(item)

In [ ]:
two_contrato.groupby(["contrato_id"])["dsp"].agg(
    [
        features.total_success_dsp5,
        features.total_success_dsp10,
        features.total_success_dsp15,
        features.total_success_dsp30,
        features.total_success_dsp60,
        features.total_success_dsp90,
    ]
).reset_index()

In [ ]:
test_list = [1, 2, 5, 90, 91, 92, 0, 90, 91, 0]

success_dsp90 = 0
for i, item in enumerate(test_list):
    if item == 90:
        new_list = test_list[i:]
        for j, item in enumerate(new_list):
            print(new_list[j])
            try:
                if new_list[j + 1] == 0:
                    print("here")
                    success_dsp90 += 1
                    break
            except IndexError as ie:
                print(ie)
                continue

In [ ]:
success_dsp90

In [ ]:
test_list[3:]